# Εργασία ταξινόμησης κειμένου

Όπως έχουμε αναφέρει, θα επικεντρωθούμε σε μια απλή εργασία ταξινόμησης κειμένου βασισμένη στο dataset **AG_NEWS**, το οποίο αφορά την ταξινόμηση τίτλων ειδήσεων σε μία από τις 4 κατηγορίες: Κόσμος, Αθλητικά, Επιχειρήσεις και Επιστήμη/Τεχνολογία.

## Το Dataset

Αυτό το dataset είναι ενσωματωμένο στο module [`torchtext`](https://github.com/pytorch/text), οπότε μπορούμε να το έχουμε εύκολα πρόσβαση.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Εδώ, `train_dataset` και `test_dataset` περιέχουν συλλογές που επιστρέφουν ζεύγη ετικέτας (αριθμός κατηγορίας) και κειμένου αντίστοιχα, για παράδειγμα:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Λοιπόν, ας εκτυπώσουμε τους πρώτους 10 νέους τίτλους από το σύνολο δεδομένων μας:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Επειδή τα σύνολα δεδομένων είναι επαναλήπτες, αν θέλουμε να χρησιμοποιήσουμε τα δεδομένα πολλές φορές πρέπει να τα μετατρέψουμε σε λίστα:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Τοκενισμός

Τώρα πρέπει να μετατρέψουμε το κείμενο σε **αριθμούς** που μπορούν να αναπαρασταθούν ως τανυστές. Αν θέλουμε αναπαράσταση σε επίπεδο λέξεων, πρέπει να κάνουμε δύο πράγματα:
* να χρησιμοποιήσουμε έναν **τοκενάιζερ** για να χωρίσουμε το κείμενο σε **τοκενς**
* να δημιουργήσουμε ένα **λεξιλόγιο** από αυτά τα τοκενς.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Χρησιμοποιώντας το λεξιλόγιο, μπορούμε εύκολα να κωδικοποιήσουμε την συμβολοσειρά που έχει υποστεί τοκενισμό σε ένα σύνολο αριθμών:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Αναπαράσταση κειμένου με Bag of Words

Επειδή οι λέξεις αντιπροσωπεύουν νόημα, μερικές φορές μπορούμε να κατανοήσουμε το νόημα ενός κειμένου απλώς κοιτάζοντας τις μεμονωμένες λέξεις, ανεξάρτητα από τη σειρά τους μέσα στην πρόταση. Για παράδειγμα, όταν ταξινομούμε ειδήσεις, λέξεις όπως *καιρός*, *χιόνι* είναι πιθανό να υποδεικνύουν *πρόγνωση καιρού*, ενώ λέξεις όπως *μετοχές*, *δολάριο* θα μπορούσαν να σχετίζονται με *οικονομικές ειδήσεις*.

Η **αναπαράσταση Bag of Words** (BoW) είναι η πιο συχνά χρησιμοποιούμενη παραδοσιακή μέθοδος αναπαράστασης με διανύσματα. Κάθε λέξη συνδέεται με έναν δείκτη διανύσματος, και το στοιχείο του διανύσματος περιέχει τον αριθμό εμφανίσεων μιας λέξης σε ένα συγκεκριμένο έγγραφο.

![Εικόνα που δείχνει πώς η αναπαράσταση διανύσματος Bag of Words αποθηκεύεται στη μνήμη.](../../../../../translated_images/el/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Μπορείτε επίσης να σκεφτείτε το BoW ως το άθροισμα όλων των διανυσμάτων one-hot-encoded για τις μεμονωμένες λέξεις του κειμένου.

Παρακάτω είναι ένα παράδειγμα για το πώς να δημιουργήσετε μια αναπαράσταση Bag of Words χρησιμοποιώντας τη βιβλιοθήκη Scikit Learn της Python:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Για να υπολογίσουμε το διάνυσμα bag-of-words από την αναπαράσταση διανυσμάτων του συνόλου δεδομένων AG_NEWS, μπορούμε να χρησιμοποιήσουμε την ακόλουθη συνάρτηση:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Σημείωση:** Εδώ χρησιμοποιούμε τη μεταβλητή `vocab_size` σε παγκόσμιο επίπεδο για να καθορίσουμε το προεπιλεγμένο μέγεθος του λεξιλογίου. Δεδομένου ότι συχνά το μέγεθος του λεξιλογίου είναι αρκετά μεγάλο, μπορούμε να περιορίσουμε το μέγεθός του στις πιο συχνές λέξεις. Δοκιμάστε να μειώσετε την τιμή του `vocab_size` και να εκτελέσετε τον παρακάτω κώδικα, και δείτε πώς επηρεάζει την ακρίβεια. Θα πρέπει να περιμένετε κάποια πτώση στην ακρίβεια, αλλά όχι δραματική, προς όφελος της υψηλότερης απόδοσης.


## Εκπαίδευση ταξινομητή BoW

Τώρα που μάθαμε πώς να δημιουργούμε την αναπαράσταση Bag-of-Words για το κείμενό μας, ας εκπαιδεύσουμε έναν ταξινομητή πάνω σε αυτήν. Πρώτα, πρέπει να μετατρέψουμε το σύνολο δεδομένων μας για εκπαίδευση με τέτοιο τρόπο ώστε όλες οι αναπαραστάσεις διανυσμάτων θέσης να μετατραπούν σε αναπαράσταση bag-of-words. Αυτό μπορεί να επιτευχθεί περνώντας τη συνάρτηση `bowify` ως παράμετρο `collate_fn` στον τυπικό torch `DataLoader`:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Τώρα ας ορίσουμε ένα απλό νευρωνικό δίκτυο ταξινόμησης που περιέχει ένα γραμμικό επίπεδο. Το μέγεθος του διανύσματος εισόδου είναι ίσο με το `vocab_size`, και το μέγεθος εξόδου αντιστοιχεί στον αριθμό των κατηγοριών (4). Επειδή λύνουμε ένα πρόβλημα ταξινόμησης, η τελική συνάρτηση ενεργοποίησης είναι η `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Τώρα θα ορίσουμε τον τυπικό βρόχο εκπαίδευσης του PyTorch. Επειδή το σύνολο δεδομένων μας είναι αρκετά μεγάλο, για τον σκοπό της διδασκαλίας μας θα εκπαιδεύσουμε μόνο για μία εποχή, και μερικές φορές ακόμη και για λιγότερο από μία εποχή (ορίζοντας την παράμετρο `epoch_size` μπορούμε να περιορίσουμε την εκπαίδευση). Θα αναφέρουμε επίσης τη συσσωρευμένη ακρίβεια εκπαίδευσης κατά τη διάρκεια της εκπαίδευσης· η συχνότητα αναφοράς καθορίζεται χρησιμοποιώντας την παράμετρο `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## Διγράμματα, Τριγράμματα και Ν-γράμματα

Ένας περιορισμός της προσέγγισης με τσάντα λέξεων είναι ότι ορισμένες λέξεις αποτελούν μέρος εκφράσεων πολλαπλών λέξεων. Για παράδειγμα, η λέξη 'hot dog' έχει εντελώς διαφορετική σημασία από τις λέξεις 'hot' και 'dog' σε άλλες περιπτώσεις. Εάν εκπροσωπούμε τις λέξεις 'hot' και 'dog' πάντα με τους ίδιους διανύσματα, μπορεί να προκαλέσει σύγχυση στο μοντέλο μας.

Για να αντιμετωπιστεί αυτό, οι **αναπαραστάσεις Ν-γραμμάτων** χρησιμοποιούνται συχνά σε μεθόδους ταξινόμησης εγγράφων, όπου η συχνότητα κάθε λέξης, διλέξης ή τριλέξης αποτελεί χρήσιμο χαρακτηριστικό για την εκπαίδευση ταξινομητών. Στην αναπαράσταση διγράμματος, για παράδειγμα, προσθέτουμε όλα τα ζεύγη λέξεων στο λεξιλόγιο, επιπλέον των αρχικών λέξεων.

Παρακάτω είναι ένα παράδειγμα για το πώς να δημιουργήσετε μια αναπαράσταση τσάντας λέξεων με διγράμματα χρησιμοποιώντας το Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Το κύριο μειονέκτημα της προσέγγισης N-gram είναι ότι το μέγεθος του λεξιλογίου αρχίζει να αυξάνεται εξαιρετικά γρήγορα. Στην πράξη, χρειάζεται να συνδυάσουμε την αναπαράσταση N-gram με κάποιες τεχνικές μείωσης διαστάσεων, όπως οι *ενσωματώσεις* (embeddings), τις οποίες θα συζητήσουμε στην επόμενη ενότητα.

Για να χρησιμοποιήσουμε την αναπαράσταση N-gram στο σύνολο δεδομένων μας **AG News**, πρέπει να δημιουργήσουμε ένα ειδικό λεξιλόγιο ngram:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Θα μπορούσαμε να χρησιμοποιήσουμε τον ίδιο κώδικα όπως παραπάνω για να εκπαιδεύσουμε τον ταξινομητή, ωστόσο, αυτό θα ήταν πολύ αναποτελεσματικό από άποψη μνήμης. Στην επόμενη ενότητα, θα εκπαιδεύσουμε έναν ταξινομητή διγράμματος χρησιμοποιώντας ενσωματώσεις.

> **Σημείωση:** Μπορείτε να κρατήσετε μόνο εκείνα τα ngrams που εμφανίζονται στο κείμενο περισσότερες φορές από τον καθορισμένο αριθμό. Αυτό θα διασφαλίσει ότι τα σπάνια διγράμματα θα παραλειφθούν και θα μειώσει σημαντικά τη διαστατικότητα. Για να το πετύχετε αυτό, ορίστε την παράμετρο `min_freq` σε υψηλότερη τιμή και παρατηρήστε την αλλαγή στο μήκος του λεξιλογίου.


## Συχνότητα Όρων Αντίστροφη Συχνότητα Εγγράφων TF-IDF

Στην αναπαράσταση BoW, οι εμφανίσεις λέξεων έχουν ίσο βάρος, ανεξάρτητα από την ίδια τη λέξη. Ωστόσο, είναι προφανές ότι οι συχνές λέξεις, όπως *ένα*, *σε*, κ.λπ., είναι πολύ λιγότερο σημαντικές για την ταξινόμηση, σε σύγκριση με εξειδικευμένους όρους. Στην πραγματικότητα, σε πολλές εργασίες NLP, ορισμένες λέξεις είναι πιο σχετικές από άλλες.

**TF-IDF** σημαίνει **συχνότητα όρων–αντίστροφη συχνότητα εγγράφων**. Είναι μια παραλλαγή του bag of words, όπου αντί για μια δυαδική τιμή 0/1 που υποδεικνύει την εμφάνιση μιας λέξης σε ένα έγγραφο, χρησιμοποιείται μια τιμή κινητής υποδιαστολής, η οποία σχετίζεται με τη συχνότητα εμφάνισης της λέξης στο σύνολο των εγγράφων.

Πιο επίσημα, το βάρος $w_{ij}$ μιας λέξης $i$ στο έγγραφο $j$ ορίζεται ως:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
όπου
* $tf_{ij}$ είναι ο αριθμός εμφανίσεων της $i$ στο $j$, δηλαδή η τιμή BoW που έχουμε δει προηγουμένως
* $N$ είναι ο αριθμός των εγγράφων στη συλλογή
* $df_i$ είναι ο αριθμός των εγγράφων που περιέχουν τη λέξη $i$ σε ολόκληρη τη συλλογή

Η τιμή TF-IDF $w_{ij}$ αυξάνεται αναλογικά με τον αριθμό των φορών που μια λέξη εμφανίζεται σε ένα έγγραφο και μειώνεται με βάση τον αριθμό των εγγράφων στη συλλογή που περιέχουν τη λέξη, κάτι που βοηθά να προσαρμοστεί το γεγονός ότι ορισμένες λέξεις εμφανίζονται πιο συχνά από άλλες. Για παράδειγμα, αν η λέξη εμφανίζεται σε *κάθε* έγγραφο της συλλογής, τότε $df_i=N$, και $w_{ij}=0$, και αυτοί οι όροι θα αγνοηθούν πλήρως.

Μπορείτε εύκολα να δημιουργήσετε την αναπαράσταση TF-IDF ενός κειμένου χρησιμοποιώντας το Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Συμπέρασμα

Παρόλο που οι αναπαραστάσεις TF-IDF παρέχουν βάρος συχνότητας σε διαφορετικές λέξεις, δεν μπορούν να αποδώσουν το νόημα ή τη σειρά. Όπως είπε ο διάσημος γλωσσολόγος J. R. Firth το 1935, «Το πλήρες νόημα μιας λέξης είναι πάντα συμφραζόμενο, και καμία μελέτη του νοήματος εκτός συμφραζομένων δεν μπορεί να ληφθεί σοβαρά υπόψη». Αργότερα στο μάθημα, θα μάθουμε πώς να αποτυπώνουμε συμφραζόμενες πληροφορίες από κείμενο χρησιμοποιώντας μοντελοποίηση γλώσσας.



---

**Αποποίηση Ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε κάθε προσπάθεια για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
